In [1]:
# import tensorflow as tf
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# learning_rate = 1e-4
# keep_prob_rate = 0.7 # 
# max_epoch = 2000
# def compute_accuracy(v_xs, v_ys):
#     global prediction
#     y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#     correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#     return result

# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)

# def bias_variable(shape):
#     initial = tf.constant(0.1, shape=shape)
#     return tf.Variable(initial)

# def conv2d(x, W):
#     # 每一维度  滑动步长全部是 1， padding 方式 选择 same
#     # 提示 使用函数  tf.nn.conv2d
    
#     return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='same')

# def max_pool_2x2(x):
#     # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
#     # 提示 使用函数  tf.nn.max_pool
    
#     return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='same')

# # define placeholder for inputs to network
# xs = tf.placeholder(tf.float32, [None, 784])/255.
# ys = tf.placeholder(tf.float32, [None, 10])
# keep_prob = tf.placeholder(tf.float32)
# x_image = tf.reshape(xs, [-1, 28, 28, 1])

# #  卷积层 1
# ## conv1 layer ##

# W_conv1 = weight_variable([7, 7, 1, 32])     # patch 7x7, in size 1, out size 32
# b_conv1 = bias_variable([32])
# h_conv1 = tf.nn.relu(conv2d(xs, W_conv1) + b_conv1)   # 卷积  自己选择 选择激活函数
# h_pool1 = max_pool_2x2(h_con1)  # 池化               

# # 卷积层 2
# W_conv2 = weight_variable([5, 5, 32, 64])     # patch 5x5, in size 32, out size 64
# b_conv2 = bias_variable([64])
# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # 卷积  自己选择 选择激活函数
# h_pool2 = max_pool_2x2(h_conv2)  # 池化

# #  全连接层 1
# ## fc1 layer ##
# W_fc1 = weight_variable([7*7*64, 1024])
# b_fc1 = bias_variable([1024])

# h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# # 全连接层 2
# ## fc2 layer ##
# W_fc2 = weight_variable([1024, 10])
# b_fc2 = bias_variable([10])
# prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


# # 交叉熵函数
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
#                                               reduction_indices=[1]))
# train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for i in range(max_epoch):
#         batch_xs, batch_ys = mnist.train.next_batch(100)
#         sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob:keep_prob_rate})
#         if i % 100 == 0:
#             print(compute_accuracy(
#                 mnist.test.images[:1000], mnist.test.labels[:1000]))

import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

# 加载MNIST数据集
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 对训练和测试图像进行归一化
train_images = train_images / 255.0
test_images = test_images / 255.0

# 将标签进行 one-hot 编码
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

learning_rate = 1e-4
keep_prob_rate = 0.7 
max_epoch = 2000

def compute_accuracy(v_xs, v_ys):
    y_pre = model(v_xs, training=False)
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    result = accuracy.numpy()
    return result

def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1, dtype=tf.float64)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape, dtype=tf.float64)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.relu(tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME'))

def max_pool_2x2(x):
    return tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义模型
class Model:
    def __init__(self):
        # 卷积层 1
        self.W_conv1 = weight_variable([7, 7, 1, 32])  # patch 7x7, in size 1, out size 32
        self.b_conv1 = bias_variable([32])
        
        # 卷积层 2
        self.W_conv2 = weight_variable([5, 5, 32, 64])  # patch 5x5, in size 32, out size 64
        self.b_conv2 = bias_variable([64])
        
        # 全连接层 1
        self.W_fc1 = weight_variable([7*7*64, 1024])
        self.b_fc1 = bias_variable([1024])
        
        # 全连接层 1
        self.W_fc2 = weight_variable([1024, 10])
        self.b_fc2 = bias_variable([10])
        
        self.trainable_variables = [self.W_conv1, self.b_conv1, self.W_conv2, self.b_conv2, \
                                   self.W_fc1, self.b_fc1, self.W_fc2, self.b_fc2]

    @tf.function
    def __call__(self, x, training):
        # 卷积层 1
        h_conv1 = conv2d(tf.expand_dims(x, axis=-1), self.W_conv1) + self.b_conv1
        h_pool1 = max_pool_2x2(h_conv1)
        
        # 卷积层 2
        h_conv2 = conv2d(h_pool1, self.W_conv2) + self.b_conv2
        h_pool2 = max_pool_2x2(h_conv2)
        
        # 全连接层 1
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, self.W_fc1) + self.b_fc1)
        h_fc1_drop = tf.nn.dropout(h_fc1, rate=1-keep_prob_rate if training else 0)
        
        # 全连接层 2
        logits = tf.matmul(h_fc1_drop, self.W_fc2) + self.b_fc2
        
        return tf.nn.softmax(logits)

# 交叉熵函数
def loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))

# 优化器
optimizer = tf.keras.optimizers.Adam(learning_rate)

# 训练步骤
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss_value = loss(labels, predictions)
    gradients = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss_value

# 创建模型
model = Model()

# 创建会话
class BreakLoop(Exception):
    pass

try:
    for epoch in range(max_epoch):
        for i in range(len(train_images) // 100):
            batch_xs = train_images[i * 100: (i + 1) * 100]
            batch_ys = train_labels[i * 100: (i + 1) * 100]
            loss_value = train_step(batch_xs, batch_ys)
    #     if epoch % 100 == 0:
            acc = compute_accuracy(test_images[:1000], test_labels[:1000])
            print("Epoch {}, Test Accuracy: {:.4f}".format(epoch, acc))
            
            if acc > 0.97:
                print('Current acc has reached {:.4f}'.format(acc))
                raise BreakLoop
except BreakLoop:
    pass



Epoch 0, Test Accuracy: 0.1170
Epoch 0, Test Accuracy: 0.1480
Epoch 0, Test Accuracy: 0.1520
Epoch 0, Test Accuracy: 0.1850
Epoch 0, Test Accuracy: 0.2260
Epoch 0, Test Accuracy: 0.2510
Epoch 0, Test Accuracy: 0.2280
Epoch 0, Test Accuracy: 0.1960
Epoch 0, Test Accuracy: 0.1930
Epoch 0, Test Accuracy: 0.2090
Epoch 0, Test Accuracy: 0.2430
Epoch 0, Test Accuracy: 0.2630
Epoch 0, Test Accuracy: 0.2550
Epoch 0, Test Accuracy: 0.2610
Epoch 0, Test Accuracy: 0.2740
Epoch 0, Test Accuracy: 0.3100
Epoch 0, Test Accuracy: 0.3550
Epoch 0, Test Accuracy: 0.3850
Epoch 0, Test Accuracy: 0.3810
Epoch 0, Test Accuracy: 0.3920
Epoch 0, Test Accuracy: 0.4220
Epoch 0, Test Accuracy: 0.4540
Epoch 0, Test Accuracy: 0.4810
Epoch 0, Test Accuracy: 0.5010
Epoch 0, Test Accuracy: 0.4850
Epoch 0, Test Accuracy: 0.4850
Epoch 0, Test Accuracy: 0.5030
Epoch 0, Test Accuracy: 0.5630
Epoch 0, Test Accuracy: 0.6150
Epoch 0, Test Accuracy: 0.6500
Epoch 0, Test Accuracy: 0.6680
Epoch 0, Test Accuracy: 0.6620
Epoch 0

Epoch 0, Test Accuracy: 0.9170
Epoch 0, Test Accuracy: 0.9140
Epoch 0, Test Accuracy: 0.9170
Epoch 0, Test Accuracy: 0.9210
Epoch 0, Test Accuracy: 0.9220
Epoch 0, Test Accuracy: 0.9260
Epoch 0, Test Accuracy: 0.9280
Epoch 0, Test Accuracy: 0.9270
Epoch 0, Test Accuracy: 0.9240
Epoch 0, Test Accuracy: 0.9280
Epoch 0, Test Accuracy: 0.9280
Epoch 0, Test Accuracy: 0.9320
Epoch 0, Test Accuracy: 0.9310
Epoch 0, Test Accuracy: 0.9340
Epoch 0, Test Accuracy: 0.9340
Epoch 0, Test Accuracy: 0.9340
Epoch 0, Test Accuracy: 0.9260
Epoch 0, Test Accuracy: 0.9230
Epoch 0, Test Accuracy: 0.9240
Epoch 0, Test Accuracy: 0.9230
Epoch 0, Test Accuracy: 0.9260
Epoch 0, Test Accuracy: 0.9270
Epoch 0, Test Accuracy: 0.9270
Epoch 0, Test Accuracy: 0.9240
Epoch 0, Test Accuracy: 0.9260
Epoch 0, Test Accuracy: 0.9310
Epoch 0, Test Accuracy: 0.9280
Epoch 0, Test Accuracy: 0.9210
Epoch 0, Test Accuracy: 0.9210
Epoch 0, Test Accuracy: 0.9230
Epoch 0, Test Accuracy: 0.9310
Epoch 0, Test Accuracy: 0.9380
Epoch 0,

Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9490
Epoch 0, Test Accuracy: 0.9480
Epoch 0, Test Accuracy: 0.9490
Epoch 0, Test Accuracy: 0.9460
Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9460
Epoch 0, Test Accuracy: 0.9460
Epoch 0, Test Accuracy: 0.9480
Epoch 0, Test Accuracy: 0.9490
Epoch 0, Test Accuracy: 0.9510
Epoch 0, Test Accuracy: 0.9530
Epoch 0, Test Accuracy: 0.9530
Epoch 0, Test Accuracy: 0.9550
Epoch 0, Test Accuracy: 0.9520
Epoch 0, Test Accuracy: 0.9520
Epoch 0, Test Accuracy: 0.9500
Epoch 0, Test Accuracy: 0.9500
Epoch 0, Test Accuracy: 0.9460
Epoch 0, Test Accuracy: 0.9410
Epoch 0, Test Accuracy: 0.9420
Epoch 0, Test Accuracy: 0.9410
Epoch 0, Test Accuracy: 0.9450
Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9480
Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9470
Epoch 0, Test Accuracy: 0.9490
Epoch 0, Test Accuracy: 0.9490
Epoch 0, Test Accuracy: 0.9480
Epoch 0, Test Accuracy: 0.9490
Epoch 0,

Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9590
Epoch 1, Test Accuracy: 0.9590
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9560
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9560
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9590
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9580
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9560
Epoch 1, Test Accuracy: 0.9550
Epoch 1, Test Accuracy: 0.9520
Epoch 1, Test Accuracy: 0.9500
Epoch 1, Test Accuracy: 0.9500
Epoch 1, Test Accuracy: 0.9500
Epoch 1, Test Accuracy: 0.9480
Epoch 1, Test Accuracy: 0.9460
Epoch 1, Test Accuracy: 0.9510
Epoch 1, Test Accuracy: 0.9510
Epoch 1, Test Accuracy: 0.9570
Epoch 1,

Epoch 1, Test Accuracy: 0.9620
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9610
Epoch 1, Test Accuracy: 0.9620
Epoch 1, Test Accuracy: 0.9640
Epoch 1, Test Accuracy: 0.9630
Epoch 1, Test Accuracy: 0.9620
Epoch 1, Test Accuracy: 0.9640
Epoch 1, Test Accuracy: 0.9650
Epoch 1, Test Accuracy: 0.9630
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9560
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9590
Epoch 1, Test Accuracy: 0.9570
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9600
Epoch 1, Test Accuracy: 0.9640
Epoch 1, Test Accuracy: 0.9640
Epoch 1, Test Accuracy: 0.9670
Epoch 1, Test Accuracy: 0.9620
Epoch 1, Test Accuracy: 0.9630
Epoch 1, Test Accuracy: 0.9670
Epoch 1, Test Accuracy: 0.9660
Epoch 1, Test Accuracy: 0.9680
Epoch 1, Test Accuracy: 0.9660
Epoch 1, Test Accuracy: 0.9670
Epoch 1, Test Accuracy: 0.9650
Epoch 1,